In [ ]:
import torch
import torch.nn as nn
import numpy as np

from models.model import Autoencoder

from utils.data import autoloader
from utils.utils import forward

from torch.utils.data import DataLoader

from torchvision.utils import make_grid

from torch.autograd import Variable


import matplotlib.pyplot as plt
%matplotlib inline

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
path = './dataset/'

A_path = path+'jaime'
B_path = path+'willyrex'

A_loader, B_loader = autoloader(A_path, B_path, bs=2, number_of_images=32)

In [ ]:
#imshow method from: https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)

In [ ]:
modelA = Autoencoder().to(device)

modelA.load_state_dict(torch.load('./checkpoints/epoch-5(A).pth'))
modelA.eval()

modelB = Autoencoder().to(device)

modelA.load_state_dict(torch.load('./checkpoints/epoch-5(B).pth'))
modelA.eval()

In [ ]:
for (A_imgs, _), (B_imgs, _) in zip(A_loader, B_loader):
    A = Variable(A_imgs).to(device)
    B = Variable(B_imgs).to(device)
    
    outputsA = forward(A, modelA, modelB, 'B2A')
    #outputsA = modelA(A)
    outputsB = modelB(B)
    
    imshow(make_grid(outputsA.detach().cpu()[0]))
    #imshow(make_grid(outputsB.detach().cpu()[0]))    